In [8]:
from src.losses import MACE_muSigma
import torch
import numpy as np

real = torch.randn(1000)
mu = torch.zeros(1000)
sigma = 1*torch.ones(1000)
y_in = 0.01*torch.cat([mu.view(-1,1), sigma.view(-1,1)], dim = 1)


MACE_muSigma(y_in, real, np.linspace(0,1,100))

tensor(0.2480)

In [ ]:
harvestor:
  early_stopped: false
  early_stopping_epoch: 0
  monitor_name: MACE
  monitor_vals: []
  training_losses: []
  val_MACE: []
  val_nll: []
  val_rmse: []
misc_info:
  input_x_shape:
  - 691
  - 8
  input_y_shape:
  - 691
  model_config:
    device: cuda
    drop_rate: 0.0
    hidden_layers:
    - 10
    - 5
    n_input: 8
    n_output: 2
  model_init: HNN
  save_path_and_name: null
  val_percentage: 0.1
training_config:
  Decay: 0.0001
  LR: 0.01
  N_Epoch: 200
  backdoor: null
  bat_size: 600
  early_stopping: true
  monitor_name: nll
  patience: 20
  train_loss: mean_std_norm_loss
  val_loss_criterias:
    MACE: MACE_muSigma
    nll: mean_std_norm_loss
    rmse: rmse_loss
  validate_times: 20
  verbose: false


In [ ]:
training_config["LR"] = LR
                training_config["bat_size"] = bat_size

                # now start training
                empty_harvestor(harvestor)

                trainer(
                    seed = SEED,
                    raw_train_X = train_X,
                    raw_train_Y = train_Y,
                    model = None,
                    training_config = training_config,
                    harvestor = harvestor,          
                    misc_info = misc_info
                )

                # now the summarizer is grepping informations from the harvestor
                sub_summarizer[(LR, bat_size)] = np.mean(harvestor["monitor_vals"][-3:])
                aid_summarizer[(LR, bat_size)] = {}
                
                for key in training_config["val_loss_criterias"].keys():
                    aid_summarizer[(LR, bat_size)]["val_"+key] = np.mean(harvestor["val_"+key][-3:])


        para_got = min(sub_summarizer.items(), key=operator.itemgetter(1))[0]
        summarizer.append(para_got)
        support_summ[para_got] = aid_summarizer[para_got]

In [ ]:
harvestor:
  early_stopped: false
  early_stopping_epoch: 0
  monitor_name: mse
  monitor_vals: []
  training_losses: []
  val_mse: []
  val_rmse: []
misc_info:
  input_x_shape:
  - 691
  - 8
  input_y_shape:
  - 691
  model_config:
    device: cuda
    hidden_layers:
    - 10
    - 5
    n_input: 8
    n_output: 1
  model_init: vanillaPred
  save_path_and_name: null
  val_percentage: 0.1
training_config:
  Decay: 0.0001
  LR: 0.01
  N_Epoch: 200
  backdoor: null
  bat_size: 600
  early_stopping: true
  monitor_name: mse
  patience: 20
  train_loss: mse_loss
  val_loss_criterias:
    mse: mse_loss
    rmse: rmse_loss
  validate_times: 20
  verbose: false


In [9]:
Z = torch.Tensor([
    [1,2,3],
    [3,4,5],
    [1,2,6],
    [1,2,6],
    [7,8,9],
    [1,6,4]
])

quants = [0.2, 0.8]



quant_bed = torch.Tensor(quants).view(-1, 1).repeat(1, len(Z)).view(-1).to(Z.device)


x_stacked = Z.repeat(len(quants), 1)
    
train_X = torch.cat([x_stacked, quant_bed.view(-1,1)], dim=1)

train_X

tensor([[1.0000, 2.0000, 3.0000, 0.2000],
        [3.0000, 4.0000, 5.0000, 0.2000],
        [1.0000, 2.0000, 6.0000, 0.2000],
        [1.0000, 2.0000, 6.0000, 0.2000],
        [7.0000, 8.0000, 9.0000, 0.2000],
        [1.0000, 6.0000, 4.0000, 0.2000],
        [1.0000, 2.0000, 3.0000, 0.8000],
        [3.0000, 4.0000, 5.0000, 0.8000],
        [1.0000, 2.0000, 6.0000, 0.8000],
        [1.0000, 2.0000, 6.0000, 0.8000],
        [7.0000, 8.0000, 9.0000, 0.8000],
        [1.0000, 6.0000, 4.0000, 0.8000]])